## Currently set to output uniform data

In [1]:
#imports
import numpy as np, pandas as pd, os, openpyxl, re

In [2]:
with open("awespd_uni.txt","r") as f:
    txt = list(filter(lambda x:not re.match(r"\s*$",x), f.read().split("\n-------end session-------\n")))

Runtime was capped at 2000ms and 5 trials of every setting done and averaged out.

In [3]:
#initializing each necessary dataframe
df1k = pd.DataFrame(columns=['nthreads','Serial','LockFree','TASLock','pthread_mutex','ALock','CLHLock'])
df1k.loc[:,'nthreads'] = [2, 3, 4, 8, 14, 28]
df1k.set_index('nthreads',inplace=True)
df1k.loc[:,:] = np.zeros((6,6))

df2k, df4k, df8k, dffair = [df1k.copy() for _ in range(4)]

dffair.drop(columns='Serial',inplace=True)

In [4]:
typedict = {-2:'Serial', -1:'LockFree', 0:'TASLock', 1:'pthread_mutex', 2:'ALock', 3:'CLHLock'}
workdict = {1000:df1k,2000:df2k,4000:df4k,8000:df8k}
modevars = {}
for j in (2,3,4,8,14,28):
    for i in range(-1,4):
        modevars[(i,j)] = []

In [5]:
#parse
for s in range(len(txt)):
    W = int(re.search(r'(?<=pkwk: )[0-9]+',txt[s]).group())
    nthreads = int(re.search(r'(?<=nsrc: )[0-9]+',txt[s]).group()) + 1
    processed = int(re.search(r"(?<=processed: )[0-9]+",txt[s]).group())
    strat = int(re.search(r'(?<=opmode: )\-*[0-9]+',txt[s]).group())
    locktype = int(re.search(r'(?<=lockmode: )\-*[0-9]+',txt[s]).group())
    if strat==-1 and locktype==-1:
        locktype = -2
    workdict[W].loc[nthreads,typedict[locktype]] += processed
    #to look at variances at W=8000 across LockFree and Awesome strategy modes
    if W==8000 and locktype>-2:
        for num in map(int,re.findall(r'(?<=Processed: )[0-9]+',txt[s])):
            modevars[(locktype,nthreads)].append(num/2000)

In [6]:
#turn into avg throughput ratio to serial, write to excel
with pd.ExcelWriter('awespdUNI.xlsx') as writer:
    for key in workdict.keys():
        workdict[key] /= 5*2000
        for j in range(1,6):
            workdict[key].iloc[:,j] /= workdict[key].iloc[:,0]
        workdict[key].to_excel(writer,sheet_name="W={}".format(key))
with pd.ExcelWriter('awefairUNI.xlsx') as writer:
    #for variance measures
    #also, since we are taking these variances across 5 samples we have var(5x), which is 25var(x)
    #so to find var(x): var(x) = var(5x)/25.
    for key in modevars:
        dffair.loc[key[1],typedict[key[0]]] = np.var(modevars[key])/25
    dffair.to_excel(writer)

In [7]:
df1k

,Serial,LockFree,TASLock,pthread_mutex,ALock,CLHLock
nthreads,,,,,,
2,393.8893,0.913644,0.910212,0.914123,0.912486,0.911876
3,409.8081,1.784006,1.812199,1.820182,1.808365,1.805725
4,410.4316,2.655666,2.748609,2.735780,2.646682,2.700777
8,410.2931,6.124649,6.201624,6.110010,5.775930,5.720964
14,410.0963,7.060083,6.439569,6.328645,6.115829,6.346217
28,409.9913,0.017246,0.192741,0.177561,0.104575,0.083975


In [8]:
df2k

,Serial,LockFree,TASLock,pthread_mutex,ALock,CLHLock
nthreads,,,,,,
2,208.7077,0.934343,0.931530,0.930585,0.934517,0.931697
3,208.4125,1.871739,1.901799,1.907087,1.901764,1.896083
4,208.1933,2.781936,2.870300,2.863872,2.853700,2.845146
8,208.5889,6.382678,6.578173,6.527603,6.346771,6.306128
14,208.7845,11.359833,11.638388,11.350971,10.948739,11.087710
28,208.8205,0.035939,0.433170,0.408356,0.340455,0.264915


In [9]:
df4k

,Serial,LockFree,TASLock,pthread_mutex,ALock,CLHLock
nthreads,,,,,,
2,104.9684,0.964887,0.965488,0.964764,0.965736,0.964433
3,104.9776,1.916410,1.952021,1.950970,1.949602,1.947748
4,105.0967,2.849854,2.930010,2.926044,2.847726,2.921090
8,105.0129,6.476748,6.774823,6.766127,6.657341,6.668989
14,105.0313,11.505035,12.294728,12.156175,11.846106,11.815731
28,103.0432,0.067549,0.977310,0.851056,0.824719,0.359198


In [10]:
df8k

,Serial,LockFree,TASLock,pthread_mutex,ALock,CLHLock
nthreads,,,,,,
2,52.7517,0.980456,0.980674,0.980560,0.980636,0.980372
3,52.8229,1.937732,1.971420,1.969494,1.969873,1.967512
4,52.7604,2.877721,2.960006,2.895531,2.889591,2.955027
8,52.6995,6.609831,6.893876,6.868807,6.827501,6.830966
14,52.8542,11.507873,12.459852,12.496727,12.395626,12.395607
28,51.5411,0.138096,1.694679,1.120964,1.120434,1.480793


In [11]:
dffair

,LockFree,TASLock,pthread_mutex,ALock,CLHLock
nthreads,,,,,
2,0.000166,0.000012,0.000445,0.000001,0.000211
3,0.000098,0.000190,0.000576,0.000321,0.000097
4,0.000403,0.000249,0.177663,0.204000,0.000761
8,0.000519,0.000805,0.002805,0.002975,0.001210
14,0.014212,0.093747,0.018317,0.009840,0.023861
28,0.000022,0.252512,0.131919,0.154342,0.247536
